In [1]:
import os
import glob
import cudf
import dask_cudf
import hydra

In [2]:
# Load hydra configuration
with hydra.initialize(version_base=None, config_path="../../../aiagents4pharma/talk2knowledgegraphs/configs"):
    cfg = hydra.compose(
        config_name="config", overrides=["tools/multimodal_subgraph_extraction=default"]
    )
    cfg = cfg.tools.multimodal_subgraph_extraction
cfg

{'_target_': 'talk2knowledgegraphs.tools.multimodal_subgraph_extraction', 'ollama_embeddings': ['nomic-embed-text'], 'temperature': 0.1, 'streaming': False, 'topk': 5, 'topk_e': 5, 'cost_e': 0.5, 'c_const': 0.01, 'root': -1, 'num_clusters': 1, 'pruning': 'gw', 'verbosity_level': 0, 'node_id_column': 'node_id', 'node_attr_column': 'node_attr', 'edge_src_column': 'edge_src', 'edge_attr_column': 'edge_attr', 'edge_dst_column': 'edge_dst', 'node_colors_dict': {'gene/protein': '#6a79f7', 'molecular_function': '#82cafc', 'cellular_component': '#3f9b0b', 'biological_process': '#c5c9c7', 'drug': '#c4a661', 'disease': '#80013f'}, 'biobridge': {'source': 'aiagents4pharma/talk2knowledgegraphs/tests/files/biobridge_multimodal/', 'node_type': ['gene/protein', 'molecular_function', 'cellular_component', 'biological_process', 'drug', 'disease']}}

In [3]:
cfg.biobridge.source

'aiagents4pharma/talk2knowledgegraphs/tests/files/biobridge_multimodal/'

In [4]:
file_list = glob.glob(os.path.join("../../../aiagents4pharma/talk2knowledgegraphs/tests/files/biobridge_multimodal/edges/", "embedding", '*.parquet.gzip'))
file_list

['../../../aiagents4pharma/talk2knowledgegraphs/tests/files/biobridge_multimodal/edges/embedding/edges.parquet.gzip']

In [5]:
edges_ddf = cudf.read_parquet(file_list[0], columns=['head_id'])
edges_ddf.shape
# edges_ddf = edges_ddf.compute()

(0, 0)

In [17]:
edges_ddf

,head_id
0,PHYHIP_(0)
1,GPANK1_(1)
2,ZRSR2_(2)
3,NRF1_(3)
4,PI4KA_(4)
...,...
1526099,positive regulation of antimicrobial humoral r...
1526100,negative regulation of transforming growth fac...
1526101,B cell receptor transport into membrane raft_(...
1526102,negative regulation of cytoskeleton organizati...


In [6]:
edges_df = cudf.concat([cudf.read_parquet(f) for f in file_list], ignore_index=True)

MemoryError: std::bad_alloc: out_of_memory: CUDA error (failed to allocate 18446744061611098112 bytes) at: /pyenv/versions/3.12.9/lib/python3.12/site-packages/librmm/include/rmm/mr/device/cuda_memory_resource.hpp:62: cudaErrorMemoryAllocation out of memory

In [6]:
# Loop over nodes and edges
graph_dict = {}
for element in ["nodes", "edges"]:
    # Make an empty dictionary for each folder
    graph_dict[element] = {}
    for stage in ["enrichment", "embedding"]:
        # print(key, path, stage)
        # Create the file pattern for the current subfolder
        file_list = glob.glob(os.path.join('../../../', 
                                           cfg.biobridge.source, 
                                           element, 
                                           stage, '*.parquet.gzip'))
        # if element != "edges" and stage == "embedding":
        # Read and concatenate all dataframes in the folder
        graph_dict[element][stage] = cudf.concat([cudf.read_parquet(f) for f in file_list], ignore_index=True)

In [12]:
import cudf
import cupy as cp
from cugraph_pyg.data import CuGraphStore, CuFeatureStore
import torch


/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/ve

ImportError: cannot import name 'CuFeatureStore' from 'cugraph_pyg.data' (/home/awmulyadi/Repositories/temp/office2/AIAgents4Pharma/venv/lib/python3.12/site-packages/cugraph_pyg/data/__init__.py)

In [11]:
graph_dict["nodes"]["embedding"]

,node_id,desc_emb,feat_emb
0,methyltransferase activity_(53517),"[0.0015654964, -0.015489323, -0.15089273, -0.0...","[-0.02258595, 0.032743175, -0.15226218, -0.010..."
1,"catalytic activity, acting on a tRNA_(53518)","[-0.008994143, -0.0030516384, -0.17048779, -0....","[0.005416816, -0.005475809, -0.17018412, -0.03..."
2,"catalytic activity, acting on DNA_(53519)","[0.0063825455, -0.004337348, -0.17087421, -0.0...","[0.02599897, -0.005063288, -0.16260663, -0.014..."
3,"catalytic activity, acting on a protein_(53520)","[0.011156816, 0.019314177, -0.15771063, 0.0061...","[0.029691847, 0.026264034, -0.15685248, -0.004..."
4,"catalytic activity, acting on RNA_(53521)","[-0.0123589905, -0.0022594568, -0.16534308, -0...","[-0.00952267, 0.020249322, -0.16449064, -0.018..."
...,...,...,...
84976,MAB21L4_(83734),"[0.0229517, -0.005922629, -0.15380856, -0.0286...","[-0.03913791850209236, -0.03291765972971916, -..."
84977,PRR23D2_(83735),"[0.0024563586, 0.024075702, -0.15282893, -0.01...","[-0.013054780662059784, 0.02872244268655777, 0..."
84978,C8orf86_(83740),"[0.023447013, -0.013911114, -0.13669135, -0.05...","[0.002801871160045266, -0.05062659829854965, 0..."
84979,CRACDL_(83746),"[0.09154602, -0.0141757075, -0.1374557, -0.084...","[-0.02955455519258976, -0.09260370582342148, -..."


In [4]:
# Prepare node dataframes by merging enrichment and embedding data
nodes_df = graph_dict["nodes"]["enrichment"].merge(
    graph_dict["nodes"]["embedding"],
    how="left",
    left_on=["node_id"],
    right_on=["node_id"],
)
nodes_df.head(5)

,node_index,node_id,node_name,node_type,desc,feat,desc_emb,feat_emb
0,76753,glutamate binding_(119024),glutamate binding,molecular_function,glutamate binding belongs to molecular_functio...,"Binding to glutamate, the anion of 2-aminopent...","[0.011298186, 0.03146643, -0.15114756, -0.0321...","[0.036096387, 0.06117738, -0.13628054, -0.0214..."
1,76754,glutamine binding_(119025),glutamine binding,molecular_function,glutamine binding belongs to molecular_functio...,"Binding to glutamine, 2,5-diamino-5-oxopentano...","[0.013914349, 0.030381806, -0.15388241, -0.025...","[0.021103695, 0.02613608, -0.16840191, -0.0396..."
2,76755,glycine binding_(119026),glycine binding,molecular_function,glycine binding belongs to molecular_function ...,"Binding to glycine, aminoethanoic acid.","[-0.008328034, 0.03096225, -0.13879329, -0.032...","[-0.012972953, 0.04233289, -0.13159795, -0.027..."
3,76756,(25S)-Delta(7)-dafachronate binding_(119027),(25S)-Delta(7)-dafachronate binding,molecular_function,(25S)-Delta(7)-dafachronate binding belongs to...,Binding to (25S)-Delta(7)-dafachronate.,"[-0.007385569, 0.025463447, -0.17066967, -0.01...","[-0.023948776, 0.032228645, -0.18667097, -0.03..."
4,76757,aspartate binding_(119028),aspartate binding,molecular_function,aspartate binding belongs to molecular_functio...,"Binding to aspartate, the alpha-amino-acid ani...","[0.026422499, 0.018216358, -0.12710637, -0.023...","[0.031808972, 0.03834121, -0.12252884, -0.0172..."


In [6]:
graph_dict["edges"]["enrichment"]

,triplet_index,head_id,tail_id,display_relation,edge_type,head_index,tail_index
0,0,PHYHIP_(0),KIF15_(8889),ppi,"[gene/protein, ppi, gene/protein]",0,8816
1,1,GPANK1_(1),PNMA1_(2798),ppi,"[gene/protein, ppi, gene/protein]",1,2787
2,2,ZRSR2_(2),TTC33_(5646),ppi,"[gene/protein, ppi, gene/protein]",2,5610
3,3,NRF1_(3),MAN1B1_(11592),ppi,"[gene/protein, ppi, gene/protein]",3,11467
4,4,PI4KA_(4),RGS20_(2122),ppi,"[gene/protein, ppi, gene/protein]",4,2117
...,...,...,...,...,...,...,...
3623251,3904603,positive regulation of antimicrobial humoral r...,ACOD1_(59926),interacts with,"[biological_process, interacts with, gene/prot...",37348,51654
3623252,3904604,negative regulation of transforming growth fac...,CD24_(34572),interacts with,"[biological_process, interacts with, gene/prot...",40247,27800
3623253,3904605,B cell receptor transport into membrane raft_(...,CD24_(34572),interacts with,"[biological_process, interacts with, gene/prot...",45323,27800
3623254,3904607,negative regulation of cytoskeleton organizati...,IQCJ-SCHIP1_(57675),interacts with,"[biological_process, interacts with, gene/prot...",35166,49927


In [27]:
edge_enrichment_df = graph_dict["edges"]["enrichment"]
edge_enrichment_df.head(5)

,triplet_index,head_id,tail_id,display_relation,edge_type,head_index,tail_index
0,0,PHYHIP_(0),KIF15_(8889),ppi,"[gene/protein, ppi, gene/protein]",0,8816
1,1,GPANK1_(1),PNMA1_(2798),ppi,"[gene/protein, ppi, gene/protein]",1,2787
2,2,ZRSR2_(2),TTC33_(5646),ppi,"[gene/protein, ppi, gene/protein]",2,5610
3,3,NRF1_(3),MAN1B1_(11592),ppi,"[gene/protein, ppi, gene/protein]",3,11467
4,4,PI4KA_(4),RGS20_(2122),ppi,"[gene/protein, ppi, gene/protein]",4,2117


In [28]:
edge_embedding_df = graph_dict["edges"]["embedding"]
edge_embedding_df.head(5)

,edge_type,edge_emb
0,"[gene/protein, ppi, gene/protein]","[0.024646243, 0.04494511, -0.13975705, -0.0281..."
1,"[drug, carrier, gene/protein]","[-0.0031494077, 0.032686442, -0.14581689, -0.0..."
2,"[drug, enzyme, gene/protein]","[0.010032998, 0.025444163, -0.1322332, -0.0111..."
3,"[drug, target, gene/protein]","[-0.011436312, 0.010051192, -0.13212629, -0.02..."
4,"[drug, transporter, gene/protein]","[0.001956201, 0.05081517, -0.1452334, -0.03340..."


In [29]:
# [gene/protein, ppi, gene/protein]

edge_enrichment_df["edge_type"] = edge_enrichment_df["edge_type"].to_arrow().to_pandas().apply(lambda x: "|".join(x))
edge_embedding_df["edge_type"] = edge_embedding_df["edge_type"].to_arrow().to_pandas().apply(lambda x: "|".join(x))

# Convert to cudf dataframes

In [31]:
# Prepare edge dataframes by merging enrichment and embedding data
edges_df = graph_dict["edges"]["enrichment"].merge(
    graph_dict["edges"]["embedding"],
    how="left",
    on="edge_type"
)
edges_df.head(5)

OverflowError: CUDF failure at: /__w/cudf/cudf/cpp/include/cudf/detail/sizes_to_offsets_iterator.cuh:327: Size of output exceeds the column size limit

In [ ]:
initial_graph["nodes"] = cudf.read_parquet(initial_graph["source"]["kg_nodes_path"])
initial_graph["edges"] = cudf.read_parquet(initial_graph["source"]["kg_edges_path"])